# LSTM

- ## Preliminaries

- ### Imports

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,TimeDistributed
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import os, os.path
from os import listdir
from os.path import isfile, join
from unicodedata import normalize
import re

Using TensorFlow backend.
/home/duc-vinh/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


- ### Check GPU usage

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [3]:
get_available_gpus()

[u'/gpu:0']

----------

- ### Check and set Twitter's API

In [4]:
import twitter
api = twitter.Api(consumer_key='LyNVanTEQEOEGKfXAMeLv6AKG',
                    consumer_secret='0lJvhaaOP5cRZWm6rxwyBIAypd1P7eiDx9f74KBDlLrSldNuBQ',
                    access_token_key='855852332034265088-geTEVmA7xIsOD3WCZyfBNnqjRdS1MhW',
                    access_token_secret='kJMwMl67e3nYrqaGWzIizxzQpRZhtBfOnwPflO1fk3cOt')

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:07.719797. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [5]:
print(api.VerifyCredentials())

{"created_at": "Sat Apr 22 18:34:31 +0000 2017", "default_profile": true, "description": "Learning how to be creative", "followers_count": 2, "friends_count": 3, "id": 855852332034265088, "lang": "en", "location": "Somewhere in the cloud", "name": "ArtistBot", "profile_background_color": "F5F8FA", "profile_banner_url": "https://pbs.twimg.com/profile_banners/855852332034265088/1492892354", "profile_image_url": "http://pbs.twimg.com/profile_images/855878764143804417/r55Z2Js5_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "TheTalkativeBot", "status": {"created_at": "Mon Apr 24 13:46:57 +0000 2017", "id": 856504737352601601, "id_str": "856504737352601601", "in_reply_to_screen_name": "dvp_tran", "in_reply_to_status_id": 856504560373911552, "in_reply_to_user_id": 747074580754403328, "lang": "en", "source": "<a href=\"http://www.google.com\" rel=\"nofollow\">TheScenarioBot</a>", "text": "@dvp_tran Automate an

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:07.777723. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


-------------------

# I. Learning from corpus

** 1. Load and convert data**

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

In [6]:
#Load and concatenate files:

DIR="../../LSTM/data/Gutenberg/ebooks-unzipped/German/"
all_files = [f for f in listdir(DIR) if isfile(join(DIR, f))]

#choose how many files to concatenate:
nb_files=7
if nb_files>len(all_files):
    nb_files=len(all_files)
    
    
out_path="german/data/"
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path+"input/"):
    os.makedirs(out_path+"input/")
    
with open(out_path+'input/german.txt', 'w') as outfile:
    for fname in all_files[0:nb_files]:
        with open(DIR+fname) as infile:
            i=0
            for line in infile:
                if i>=50:
                    outfile.write(line)
                i=i+1
        print ("Done concatenating file : %s" %fname)

Done concatenating file : 2146-8.txt
Done concatenating file : 2779-8.txt
Done concatenating file : 2409-8.txt
Done concatenating file : 4013-8.txt
Done concatenating file : 2947-8.txt
Done concatenating file : 3498-8.txt
Done concatenating file : 2314-8.txt


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:08.169241. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [7]:
#load file
file_name=out_path+'input/german.txt'
text = open(file_name).read()
text=normalize('NFKD',text.decode('latin1')).encode('ASCII', 'ignore')

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:08.206426. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [8]:
#text = text.replace(to_delete,"").replace('Digitized by',"").replace('Google',"") 
text = re.sub("\n\n+" , "\n", text)

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:08.367382. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [9]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print('total chars:',VOCAB_SIZE)

corpus length: 2215861
total chars: 94


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:08.413013. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


**Warning:** The RNN takes in input numerical data hence the necessity to convert strings into numerical values.

In [10]:
#creating mapping between indexes and characters
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:08.515444. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


We’re gonna use Keras to create and train our Network, so we must convert the data into this form: (number_of_sequences, length_of_sequence, number_of_features).
- nb of features = length of the char array
- length of sequence = batch size
- nb of sequence = len(data) divided by batch size.

**Warning : ** target sequence is setted by shifting the source/input sequence by one character with both having the same length.

In [11]:
%%time

SEQ_LENGTH=100
#Build three dimensional arrays
X = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #input
y = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #target

#Build sequences
for i in range(0, len(text)/SEQ_LENGTH):
    X_sequence = text[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_indices[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_indices[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

CPU times: user 1.98 s, sys: 140 ms, total: 2.12 s
Wall time: 2.11 s


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:08.527590. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


** 2. Build the network**

In [12]:
HIDDEN_DIM= 500 #500
LAYER_NUM = 2


model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:10.700373. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [13]:
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:12.687738. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [14]:
# Generate some sample before training to know how bad it is!
bla = generate_text(model, 100, VOCAB_SIZE, indices_char)
#api.PostUpdate(status=bla[0:123])

MhhqqL4LLLLFK99U44U2A''''>''llGGGjj??????772$$$$3333WW11&&&S888""##6;;;llFF4UUUWWWd%G$ "C***__''''''

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:12.699865. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


**3. Train network**

In [15]:
def get_iternb(string):
    return re.findall(r'checkpoint_500_epoch_(.*).hdf5', string)[0]

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:18.501295. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [16]:
#batch size equals to seq length here
BATCH_SIZE=100
#len of desired output
GENERATE_LENGTH=140
DIR=out_path+"weights/weight_attempt_s02/"
flag=True

try:
    onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]
    iteration=[]
    for files in onlyfiles:
        iteration.append(int(get_iternb(files)))
    iteration=max(iteration)

    last_checkpoint=DIR+onlyfiles[0][0:21]+str(iteration)+'.hdf5'
except Exception as e:
    print(e)
    onlyfiles=[]
    if not os.path.exists(DIR):
        os.makedirs(DIR)
    nb_files=0

if nb_files>0:
    model.load_weights(last_checkpoint)
else:
    iteration=0
    
print("Starting at iteration : %s" %iteration)
while flag==True:
    print('\n')
    print('-'*20)
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=2, nb_epoch=1)
    iteration += 1
    bla=generate_text(model, GENERATE_LENGTH,VOCAB_SIZE, indices_char)
    if iteration % 10 == 0:
        print("\n\nIteration nb : %s" %iteration)
        #api.PostUpdate(status=bla[0:123])
        model.save_weights(DIR+'checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, iteration))
        #remove unecessary files:
        for files in onlyfiles:
            try:
                if files:
                    os.remove(DIR+files)
            except:
                pass
        onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]

    if iteration>=500:
        print("Stopping...")
        flag=False

/home/duc-vinh/anaconda2/lib/python2.7/site-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


[Errno 2] No such file or directory: 'german/data/weights/weight_attempt_s02/'
Starting at iteration : 0


--------------------
Epoch 1/1
407s - loss: 2.7859
< einer under der under Sohlen die Sohlen die Sohlen die Sohlen die Sohlen die Sohlen die Sohlen die Sohlen die Sohlen die Sohlen die Sohlen

--------------------
Epoch 1/1
403s - loss: 2.0011
!  Die sich seine Schlichen und den Gescheiten und den Gescheiten und den Gescheiten und den Gescheiten und den Gescheiten und den Gescheite

--------------------
Epoch 1/1
403s - loss: 1.7291
her der Start und Standen und seiner Standen zu seinen sollten war, da er sich selbst der Starten und seiner Standen zu seinen sollten war, 

--------------------
Epoch 1/1
403s - loss: 1.5601
Or der Geschichten der Gestalt der Geister der Geschichten der Gestalt der Geister der Geschichten der Gestalt der Geister der Geschichten d

--------------------
Epoch 1/1
403s - loss: 1.4503
Sten und die Schafe der Schafe, und die Schafe der Schafe, der sich in

Es schneider aus den To

--------------------
Epoch 1/1
402s - loss: 0.5275
?  Warum ist alles weiter nichts als diesen kleinen
Vorteil ist geglabt, den Sohn der Sitten des Aufressenheit im Gegenteilk
tuchtig schau

--------------------
Epoch 1/1
402s - loss: 0.5168
y to the etext collections, use FTP or any
Web browser to visit a Project Gutenberg mirror (mirror
sich auch not allob disclaimers of impl

--------------------
Epoch 1/1
402s - loss: 0.5069
6. Und der Herr der Geister hat ihn begabtet einen Tag der Freiheit?
10. Und nun horen wir uns auf der Erde. Und es wird die Handel der Erd

--------------------
Epoch 1/1
402s - loss: 0.4979
Project Gutenberg Etextes "Michated
       thin that scannires etext endibald or leysed to whilling the mexinext cived etext on
the notile

--------------------
Epoch 1/1
403s - loss: 0.4890
x.  Da legte sich nicht, es ist mir den gewissen
Schriftstellern zu besinden, nachdem der Hof und den Mund, und er wird gesehen werden alle

------------------

11. Und der Herr der Geister wird Mitzen wisd und sich und sagte Tor, weil sie voller Zeichen ist in

--------------------
Epoch 1/1
402s - loss: 0.3259
$
] OR MERCH DRMANEY OF DACHAL
IF YOU DON'T HAVE TO?
Project Gutenberg is a PReJITHIV.

Project Gutenberg Literard
Ar per our News/inge

--------------------
Epoch 1/1
402s - loss: 0.3244
7. Und es wurde geboren in ihrer Mittel, welche fahig sind, und vor ihm allerdei
hinwegen.
24. Die Weisheit tretzt man vor alle Dinge dein

--------------------
Epoch 1/1
402s - loss: 0.3227
7. Und erheben wird sein Haus zu verhoren uberloschen, das eben zu
ruhen; und ich bitte aus, ihr kunstlich abgefuhrt war.


LIc

Mein 

--------------------
Epoch 1/1
402s - loss: 0.3205
Geschichte ist.  Was mussen Sie
sonst gegen dieses sogleich nicht zu erblicken hatten?--Nun, das starkete und weiter ins Methus
aufrugen, 

--------------------
Epoch 1/1
402s - loss: 0.3189
verletzten; ich bin uberzeugt, bei welchem
Agathon, sein Bestes spater getroffen wurden, d

402s - loss: 0.2728
; sie gestehen, da er sich,
da ich sie nach meine Stelle sehen lie; und das ist, alles
aufzuopfen ubrig bin uberwahit zu werden, welche er

--------------------
Epoch 1/1
403s - loss: 0.2732
"Dergleichen Inschriften, scheint es, hat der
Oheim von den Vertraulichkeiten selbst, und zwar auf
Absichten und Sklaven gesetzt hatte.  S

--------------------
Epoch 1/1
403s - loss: 0.2716
3, 92, 92, 91 or 90

Just searind a cho of the money havade of contribution
you can think of.  Money should warent od y the menthel to ge

Iteration nb : 130


--------------------
Epoch 1/1
402s - loss: 0.2708
"Defects".  Among other
things, Defects may take the form of softer
if theyerses lick) sonderging not all dustcendd
ot ze spoll on       

--------------------
Epoch 1/1
403s - loss: 0.2702
Geschichts-zu machenlich zu tun sich
um eine Person, welche begriff bei diesen Privat-Verhaltnis geliebtes Kart
genug zweifelhaft abzulege

--------------------
Epoch 1/1
402s - loss: 0.2693
Bescha

...

Glanz den Nachsten der Strahlen, in 

Iteration nb : 340


--------------------
Epoch 1/1
402s - loss: 0.2007
~
2: Or a mont provitar to guten it any (onnerpestraf Diskys of this etext if you want to.

*BEFORE!* YOU USE OR READ THIS ETEXT
By using

--------------------
Epoch 1/1
402s - loss: 0.2004
Er sprach nur das Allernotigste,
Liege, gulten soll mich zu, und war
Unterricht und Last und Arbbystes Nahrundsche;
Da krautverleihlt ins

--------------------
Epoch 1/1
402s - loss: 0.2007
) dem menschlichen Geschlechtern
hatten Manner und Freude uber eine Liebhaber ganzlich von den
Feindangen. Ein bloeres Schweig, sondern vo

--------------------
Epoch 1/1
402s - loss: 0.2000
.  Er suche
Arbeit, er suche Hilfe, er suche Recht, so wird er jedem
andern nachgelassen wie dum, indem ich mich daruber zu fragen, welche

--------------------
Epoch 1/1
402s - loss: 0.2002
?

"Falsch ist nicht, in grassermaigen Herren:
Wenn man ein solches Flei nicht segner
Schlaf geschicktet die groste Hut,
Der Sch

____________________________________________________________________________________________________________________________________________

--------------------
Epoch 1/1
402s - loss: 0.1950
Qualen fur
mich zu ersinnen hat, ich wurde durch die Unbringenzeit besser schicke.  Denn was
vorler Jedechter, wie man sie gisse zu den Ei

--------------------
Epoch 1/1
402s - loss: 0.1946
ber einem anderen
Leute betrachtet, wahnsinniger zu fragen, ob Sie nicht
bekommt, wird nie ein lebermals abgelegene Teil ist wert; sofort 

--------------------
Epoch 1/1
402s - loss: 0.1948
<haben.  Man braucht nur
einen Schrecknern auf dem Lande, und der "Wagen nicht hat er beschadigt und geht
haben, in einen Erzahlung zu sei

--------------------
Epoch 1/1
402s - loss: 0.1946
's pochet in sie
  erluste--                   Rudolucf wohl gar noch heut,)
Und die ihr klingt end offen schildert.

                  

--------------------
Epoch 1/1
402s - loss: 0.1938
90 through etext99 or etext07 reerare etext i

at hour the manthro Strenet, antere findenwor

--------------------
Epoch 1/1
402s - loss: 0.1896
" von Heinrich Heine

aus dem Gefahr Mani ausgesehen, meine besten Anordnu glanzende
Schwierigkeiten, es meine eigene Wirkung gehort, so 

--------------------
Epoch 1/1
403s - loss: 0.1895
hen, welche eben das ernsthafte und stellte sich das groe
Entfernheitswart, dessen Leben war und an der es war, malte sie stoden, da er nic

--------------------
Epoch 1/1
402s - loss: 0.1893
sen, wenn eine Wohnung und Barbareisen zu verbergen, je er
wechselde auch nicht ein unruhm.  Da er mir die Tante in einen
bessern Zustand 

--------------------
Epoch 1/1
402s - loss: 0.1890


Sekretar.  Eurem Befehl zu gehorchen, wart' ich die, welche
man sieht, was kein Frieder bringen konnen, werden sich bewegen sein, welche

--------------------
Epoch 1/1
402s - loss: 0.1885
Liebsten Ausgebickt.  Die Liebe ist nicht zu wichtigen pflegte, seine
Erbitterung fuhrte, die Blatter ist, wenn ihr Ihr nicht mehr bruhalt

KeyboardInterrupt: 

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-25 04:18:18.506630. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


**4. Generate text**

In [ ]:
def save_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    if not os.path.exists(out_path+"generate/"):
        os.makedirs(out_path+"generate/")
    with open(out_path+"generate/output.txt","w") as f:
        f.write(('').join(y_char))
    return ('').join(y_char)


In [ ]:
#seed with particular text:
def generate_text_seeded(model,seed,length, vocab_size, ix_to_char):
    # starting with random character
    # char_indices
    ix = [char_indices[x] for x in seed]
    y_char = [x for x in seed]
    X = np.zeros((1, length, vocab_size))
    for i in range(len(ix)) :
        X[0, i, :][ix[i]] = 1
        print(ix_to_char[ix[i]], end="")
    to_substract = len(ix)
    for i in range(length-to_substract):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [ ]:
generate_text_seeded(model,normalize('NFKD',"ich bin ".decode('latin1')), 100, VOCAB_SIZE, indices_char)

In [ ]:
%%time
out = save_text(model, 1500, VOCAB_SIZE, indices_char)